In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(table)
setTable(tableRt, tableType = "RESOURCE_TABLE")

Set Redshift endpoint done. 2017-03-17T11:24:22


"beacons_4744_rt"

In [2]:
# Packages

include("../../Lib/Include-Package-v2.1.jl")


In [3]:
customer = "Nat Geo" 
productPageGroup = "Nat Geo Homepage" # primary page group 
localTable = "$(table)_DOC_view"
localTableRt = "$(tableRt)_DOC_view"
localTableRt = "$(tableRt)_URL_view_cd"
localUrl = "%www.nationalgeographic.com/"
localUrlFull = "http://www.nationalgeographic.com/"

"http://www.nationalgeographic.com/"

In [4]:
# v92 code


timeString = "$(padDateTime(startTimeTest)) to $(padDateTime(endTimeTest))"


LoadError: LoadError: UndefVarError: startTimeTest not defined
while loading In[4], in expression starting on line 4

In [5]:
firstAndLast = getBeaconsFirstAndLast()
endTime = DateTime(firstAndLast[1,2])
startTime = DateTime(endTime - Day(1))

startTimeMs = datetimeToMs(startTime)
endTimeMs = datetimeToMs(endTime)

#graphics covering 24 hours or less can use minutes, otherwise try :hours or :days or :seconds depending on how wide the time range is
datePart = :minute
timeString = "$(padDateTime(startTime)) to $(padDateTime(endTime))"
println(timeString)

March 15, 2017 23:59:59 to March 16, 2017 23:59:59


In [6]:
try
    query("""\
        drop view if exists $localTable
    """)

     query("""\
        create or replace view $localTable as 
            (
            select * from $table where 
                "timestamp" between $startTimeMs and 
                $endTimeMs and 
                page_group = '$(productPageGroup)' and
                params_u = '$(localUrlFull)'
                limit 20000
            )
        """)

    # Some routines use the unload events, some do not.  First count is all beacons such as page view and unload
    # where beacon_type = 'page view'
    cnt = query("""SELECT count(*) FROM $localTable""")
    #Hide output from final report
    println("$localTable count is ",cnt[1,1])
catch y
    println("Setup Tables Exception ",y)
end


beacons_4744_DOC_view count is 20000


In [7]:
displayTitle(chart_title = "Top URL Page Views for $(productPageGroup)", chart_info = [timeString],showTimeStamp=false)

query("""\

select count(*),params_u
FROM $(localTable)
where 
beacon_type = 'page view' 
group by params_u
order by count(*) desc
limit 5

""")

Top URL Page Views for Nat Geo Homepage March 15, 2017 23:59:59 to March 16, 2017 23:59:59

,count,params_u
1,18008,http://www.nationalgeographic.com/


In [8]:
displayTitle(chart_title = "Top URL Page Views for $(productPageGroup)", chart_info = [timeString],showTimeStamp=false)

query("""\

select count(*),session_id,params_u
FROM $(localTable)
where 
beacon_type = 'page view' and
params_u ilike '$(localUrl)'
group by params_u,session_id
order by count(*) desc
""")

Top URL Page Views for Nat Geo Homepage March 15, 2017 23:59:59 to March 16, 2017 23:59:59

,count,session_id,params_u
1,596,NA,http://www.nationalgeographic.com/
2,177,917f08f4-41b2-46fe-bb19-8c4dcf7b2fa0-omrjpv,http://www.nationalgeographic.com/
3,17,16d9b5d7-e314-4a1f-a94a-aba878b3873e-omx2p0,http://www.nationalgeographic.com/
4,15,aaef1dfb-8574-4829-ae1d-8fcac9fd3be1-omwkql,http://www.nationalgeographic.com/
5,15,682a6025-8b6d-47e8-b9dd-4a523bab42a6-omvymi,http://www.nationalgeographic.com/
6,15,e80a3e11-b578-4622-90f6-b6c69158ad3a-omvsy0,http://www.nationalgeographic.com/
7,15,0ddc69ab-6e90-432c-8fe9-0d7ddedac0ba-omwqzj,http://www.nationalgeographic.com/
8,15,215dc602-4147-48cd-a979-39bb6d650583-omvxdc,http://www.nationalgeographic.com/
9,15,10c6466c-eb44-4a37-ac3a-9340bef731ce-omxfhm,http://www.nationalgeographic.com/
10,15,403f603f-1885-4808-ba2f-09015e3d7946-omx63z,http://www.nationalgeographic.com/


In [9]:
sessionId = "ad2fd687-691f-4764-a9bb-2182db03634e-oho76h"

"ad2fd687-691f-4764-a9bb-2182db03634e-oho76h"

In [10]:
displayTitle(chart_title = "Top URL Page Views for $(productPageGroup)", chart_info = [timeString],showTimeStamp=false)

query("""\

select count(*),session_id,params_u,"timestamp"
FROM $(localTable)
where 
beacon_type = 'page view' and
session_id = '$sessionId'
group by params_u,session_id,"timestamp"
order by "timestamp" asc
""")

Top URL Page Views for Nat Geo Homepage March 15, 2017 23:59:59 to March 16, 2017 23:59:59

""


In [11]:
ts = "('1482106711154','1482106711161','1482107709775')";

In [12]:
    query("""\
        drop view if exists $localTableRt
    """)



#query("""create or replace view $localTableRt as (select * from $tableRt where "timestamp" between $startTimeMs and $endTimeMs and (url ilike '$(localUrlRt)' or params_u ilike '$(localUrlRt)'))""")
query("""create or replace view $localTableRt as (select * from $tableRt where "timestamp" between $startTimeMs and $endTimeMs and session_id = '$sessionId') limit 10000""")

# Some routines use the unload events, some do not.  First count is all beacons such as page view and unload
# where beacon_type = 'page view'
cnt = query("""SELECT count(*) FROM $localTableRt""")
#Hide output from final report
println("$localTableRt count is ",cnt[1,1])


beacons_4744_rt_URL_view_cd count is 0


In [13]:
sBeacon = query("""\
select count(*)
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
""")
display(sBeacon)

sRt = query("""\
select count(*)
from $localTableRt
where session_id = '$(sessionId)' and "timestamp" in $(ts)
""")


,count
1,0


,count
1,0


In [14]:
#methods(beautifyDF)
#methodswith(DataArray)
#describe(tlfields)

In [15]:
#tlfields[:, Bool[ismatch(r"^session_", x) for x in tlfields[:column]]]
#tlfields[:, :Bool[ismatch(r"^session_", x) for x in tlfields[1:1,:]]]
#beautifyDF(tlfields[1:1,:session_id])

#println(tlfields[1:1,:session_id])
#println(tlfields[1:1,:session_isunload])
#println(tlfields[1:1,:session_latest])
#println(tlfields[1:1,:session_obopages])
#println(tlfields[1:1,:session_pages])
#println(tlfields[1:1,:session_start])
#println(tlfields[1:1,:session_totalloadtime])


In [16]:
displayTitle(chart_title = "Top Level Fields (1) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
domain,"timestamp",key,http_method,http_referrer,http_version,site_version,url
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Top Level Fields (1) from Beacon

""


In [17]:
displayTitle(chart_title = "Top Level Fields (2) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
beacon_type,page_group,remote_ip,proxy_address,
errors,warnings,spdy,ssl,ipv6,
mobile_connection_type,compression_types,ab_test
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Top Level Fields (2) from Beacon

""


In [18]:
displayTitle(chart_title = "Session from Beacon", showTimeStamp=false)
sessionFields = query("""\
select session_id,session_start,session_latest,session_obopages,session_pages,session_totalloadtime,session_isunload
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Session from Beacon

""


In [19]:
displayTitle(chart_title = "User Agent Fields from Beacon", showTimeStamp=false)
sessionFields = query("""\
select user_agent_family,user_agent_major,user_agent_minor,user_agent_mobile,user_agent_model,user_agent_os,user_agent_osversion,
user_agent_manufacturer,user_agent_device_type,user_agent_isp,params_ua_plt,params_ua_vnd
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

User Agent Fields from Beacon

""


In [20]:
displayTitle(chart_title = "User Agent Fields Raw from Beacon", showTimeStamp=false)
sessionFields = query("""\
select user_agent_raw
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

User Agent Fields Raw from Beacon

""


In [21]:
displayTitle(chart_title = "Geo from Beacon", showTimeStamp=false)
sessionFields = query("""\
select geo_cc,geo_city,geo_lat,geo_lon,geo_netspeed,geo_org,geo_postalcode,geo_rg,geo_isp
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Geo from Beacon

""


In [22]:
displayTitle(chart_title = "Bandwidth from Beacon", showTimeStamp=false)
sessionFields = query("""\
select bandwidth_kbps,bandwidth_error_pc,bandwidth_block	
from $localTable
where 
session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
--and bandwidth_kbps IS NOT NULL
""")

Bandwidth from Beacon

""


In [23]:
displayTitle(chart_title = "Timers (T) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
timers_t_resp,timers_t_page,timers_t_done,timers_t_domloaded,timers_t_configfb,timers_t_configjs,
timers_t_load,timers_t_prerender,timers_t_postrender
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""");
display(sessionFields)
println("times_t_resp = Beacon Back End")
println("times_t_page = Beacon Front End")
println("times_t_done = Beacon Page Load")

Timers (T) from Beacon

""


times_t_resp = Beacon Back End
times_t_page = Beacon Front End
times_t_done = Beacon Page Load


In [24]:
displayTitle(chart_title = "Timers (boomr) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
timers_boomr_ld,timers_boomr_fb,timers_boomr_lat,timers_boomerang,timers_fb_to_boomr,timers_navst_to_boomr,timers_boomr_to_end
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Timers (boomr) from Beacon

""


In [25]:
displayTitle(chart_title = "Timers from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
timers_before_dns,timers_dns,timers_tcp,timers_ssl,timers_domload,timers_domready,timers_renderstart,timers_loaded,timers_missing
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""");
display(sessionFields)
println("timers_dns = DNS")
println("timers_tcp = TCP")
println("timers_ssl = SSL")
println("timers_domload = DOM Loading")
println("timers_domready = DOM Complete")
println("timers_renderstart = First Paint")
println("timers_loaded = nt_load_end - nt_nav_st")

Timers from Beacon

""


timers_dns = DNS
timers_tcp = TCP
timers_ssl = SSL
timers_domload = DOM Loading
timers_domready = DOM Complete
timers_renderstart = First Paint
timers_loaded = nt_load_end - nt_nav_st


In [26]:
displayTitle(chart_title = "Timers from Beacon", showTimeStamp=false)
sessionFields = query("""\
select timers_custom0,timers_custom1,timers_custom2,timers_custom3,timers_custom4,timers_custom5,timers_custom6,timers_custom7,timers_custom8,timers_custom9	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""");
display(sessionFields)
println("timers_custom3 = Moat Ad")
println("timers_custom4 = DoubleClick")
println("timers_custom5 = JS")
println("timers_custom6 = CSS")
println("timers_custom7 = JPG")

Timers from Beacon

""


timers_custom3 = Moat Ad
timers_custom4 = DoubleClick
timers_custom5 = JS
timers_custom6 = CSS
timers_custom7 = JPG


In [27]:
displayTitle(chart_title = "Custom from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
custom_metrics_0,custom_metrics_1,custom_metrics_2,custom_metrics_3,custom_metrics_4,custom_metrics_5,custom_metrics_6,custom_metrics_7,custom_metrics_8,custom_metrics_9	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Custom from Beacon

""


In [28]:
displayTitle(chart_title = "CloudFlare Headers from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
headers_cf_visitor,headers_cf_ray,headers_cf_connecting_ip,headers_x_forwarded_for,headers_x_forwarded_proto
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

CloudFlare Headers from Beacon

""


In [29]:
displayTitle(chart_title = "HTTP Headers from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
headers_connection,headers_host,headers_accept_encoding,headers_accept_language,headers_accept,headers_content_length,headers_various	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

HTTP Headers from Beacon

""


In [30]:
displayTitle(chart_title = "Navigation Timing (1) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
params_nt_nav_type,params_nt_red_cnt,params_nt_spdy,params_nt_cinf
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Navigation Timing (1) from Beacon

""


In [31]:
displayTitle(chart_title = "Navigation Timing (2) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
params_nt_nav_st,
params_nt_red_st,params_nt_red_end,(params_nt_red_end-params_nt_red_st) red_delta,
params_nt_fet_st,
params_nt_dns_st,params_nt_dns_end,(params_nt_dns_end-params_nt_dns_st) dns_delta,
params_nt_con_st,params_nt_ssl_st,params_nt_con_end,(params_nt_con_end-params_nt_con_st) con_delta
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Navigation Timing (2) from Beacon

""


In [32]:
displayTitle(chart_title = "Navigation Timing (3) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
params_nt_req_st,params_nt_res_st,
params_nt_unload_st,params_nt_unload_end,
params_nt_first_paint,params_nt_domloading,
params_nt_res_end
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Navigation Timing (3) from Beacon

""


In [33]:
displayTitle(chart_title = "Navigation Timing (4) from Beacon", showTimeStamp=false)
sessionFields = query("""\
select 
params_nt_domint,params_nt_domcontloaded_st,params_nt_domcontloaded_e,params_nt_domcomp,
params_nt_load_st,params_nt_load_end
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Navigation Timing (4) from Beacon

""


In [34]:
displayTitle(chart_title = "Boomerang Debug Info from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_rt_bmr_conen,params_rt_bmr_const,params_rt_bmr_domen,params_rt_bmr_domst,params_rt_bmr_fetst,params_rt_bmr_reqst,params_rt_bmr_resen,params_rt_bmr_resst,params_rt_bmr_secst
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Boomerang Debug Info from Beacon

""


In [35]:
displayTitle(chart_title = "Config.js Debug Info from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_rt_cnf_conen,params_rt_cnf_const,params_rt_cnf_domen,params_rt_cnf_domst,params_rt_cnf_fetst,params_rt_cnf_reqst,params_rt_cnf_resen,params_rt_cnf_resst,params_rt_cnf_secst
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Config.js Debug Info from Beacon

""


In [36]:
displayTitle(chart_title = "Debug Info from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_rt_abld,params_rt_blstart,params_rt_bstart,params_rt_cstart,params_rt_end,params_rt_ntvu,params_rt_obo,
params_rt_quit,params_rt_sh,params_rt_si,params_rt_sl,params_rt_srst,params_rt_start,params_rt_tstart,params_rt_tt,params_rt_ss,
params_cmet_mpulseid,params_errors,params_h_t,params_if,params_v,params_h_cr
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Debug Info from Beacon

""


In [37]:
displayTitle(chart_title = "URL from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_u,params_pgu,params_r,params_r2,params_nu	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

URL from Beacon

""


In [38]:
displayTitle(chart_title = "Page Structure from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_dom_doms,params_dom_img,params_dom_img_ext,
params_dom_script,params_dom_script_ext,
params_dom_ln,params_dom_res,params_dom_sz,params_dom_res_slowest
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""");
display(sessionFields)
println("params_dom_doms\t\t=\tUnique Domain Count")
println("params_dom_img\t\t=\tDOM Images On Page")
println("params_dom_img_ext\t=\tDOM Images Ext")
println("params_dom_script\t=\tDOM Srcipts On Page")
println("params_dom_script_ext\t=\tDOM Scripts Ext")
println("params_dom_ln\t\t=\tDOM Nodes On Page")
println("params_dom_res\t\t=\tResources in RT")
println("params_dom_sz\t\t=\tDOM Size")


Page Structure from Beacon

""


params_dom_doms		=	Unique Domain Count
params_dom_img		=	DOM Images On Page
params_dom_img_ext	=	DOM Images Ext
params_dom_script	=	DOM Srcipts On Page
params_dom_script_ext	=	DOM Scripts Ext
params_dom_ln		=	DOM Nodes On Page
params_dom_res		=	Resources in RT
params_dom_sz		=	DOM Size


In [39]:
displayTitle(chart_title = "Screen and Device Details from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_scr_bpp,params_scr_dpx,params_scr_mtp,params_scr_orn,params_scr_xy,params_mem_total,params_mem_used,params_bat_lvl,params_cpu_cnc	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Screen and Device Details from Beacon

""


In [40]:
displayTitle(chart_title = "Visibility State from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_vis_lh,params_vis_lv,params_vis_st	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Visibility State from Beacon

""


In [41]:
displayTitle(chart_title = "XHR Details from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
http_errno,params_http_method,params_http_hdr,params_http_initiator,params_xhr_sync,params_rt_subres
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

XHR Details from Beacon

""


In [42]:
displayTitle(chart_title = "Bandwidth & Latency Details from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_bw_time,params_lat,params_lat_err
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Bandwidth & Latency Details from Beacon

""


In [43]:
displayTitle(chart_title = "Mobile Connection Details from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_mob_ct,params_mob_bw,params_mob_mt	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Mobile Connection Details from Beacon

""


In [44]:
displayTitle(chart_title = "Params Custom from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
params_custom0_st,params_custom1_st,params_custom2_st,params_custom3_st,params_custom4_st,params_custom5_st,params_custom6_st,params_custom7_st,params_custom8_st,params_custom9_st	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Params Custom from Beacon

""


In [45]:
displayTitle(chart_title = "Google Analytics from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
ga_clientid,ga_utm_source,ga_utm_medium,ga_utm_term,ga_utm_content,ga_utm_campaign	
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Google Analytics from Beacon

[ODBC] 42703: ERROR: column "ga_clientid" does not exist in beacons_4744_doc_view;
Error while executing the query


LoadError: LoadError: [ODBC]: SQLExecDirect failed; Return Code: SQL_DEFAULT_ERROR
while loading In[45], in expression starting on line 2

In [46]:
displayTitle(chart_title = "AA & IA from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
aa_aid,aa_mid,aa_campaign,ia_coreid,ia_mmc_vendor,ia_mmc_category,ia_mmc_placement,ia_mmc_item
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

AA & IA from Beacon

[ODBC] 42703: ERROR: column "aa_aid" does not exist in beacons_4744_doc_view;
Error while executing the query


LoadError: LoadError: [ODBC]: SQLExecDirect failed; Return Code: SQL_DEFAULT_ERROR
while loading In[46], in expression starting on line 2

In [47]:
displayTitle(chart_title = "Customer Dimensions from Beacon", showTimeStamp=false)
sessionFields = query("""\
select
cdim
from $localTable
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp"
""")

Customer Dimensions from Beacon

""


***
## RT Section
***

In [48]:
displayTitle(chart_title = "Matching Records from RT Data", showTimeStamp=false)
sessionFields = query("""\
select
count(*),"timestamp", session_id
from $localTableRt
where session_id = '$(sessionId)' and "timestamp" in $(ts)
group by session_id,"timestamp"
order by "timestamp"
""")

Matching Records from RT Data

""


In [49]:
displayTitle(chart_title = "Description Fields from RT Data", showTimeStamp=false)
sessionFields = query("""\
select
session_id,session_start,"timestamp",params_u
from $localTableRt
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp",start_time
limit 1000
""")

display(sessionFields[1:3,:])
#display(sessionFields[31:end,:])
#data is all the same except timestamp may vary depending on in clause above

Description Fields from RT Data

""


In [50]:
displayTitle(chart_title = "URL Fields from RT Data", showTimeStamp=false)

#todo for loop on all in the "in" list
#figure out how to do more than 30

sessionFields = query("""\
select
initiator_type, substring(url for position('/' in substring(url from 9)) +7) urlgroup,url
from $localTableRt
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp",start_time
limit 1000
""")

display(sessionFields[1:30,:])
display(sessionFields[31:end,:])

URL Fields from RT Data

""


""


In [51]:
displayTitle(chart_title = "Timing from RT Data", showTimeStamp=false)
sessionFields = query("""\
select
start_time,
redirect_start,redirect_end,
fetch_start,
dns_start,dns_end,
tcp_connection_start,secure_connection_start,
tcp_connection_end,
request_start, response_first_byte,response_last_byte,
worker_start
from $localTableRt
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by "timestamp",start_time
limit 1000
""")
display(sessionFields[1:30,:])
display(sessionFields[31:end,:])

Timing from RT Data

""


""


In [52]:
displayTitle(chart_title = "Additional Details from RT Data", showTimeStamp=false)
sessionFields = query("""\
select
encoded_size,transferred_size,decoded_size,height,width,x,y
from $localTableRt
where session_id = '$(sessionId)' and "timestamp" in $(ts)
order by start_time
limit 1000
""")

Additional Details from RT Data

""


In [53]:
try
    query("""\
        drop view if exists $localTable
    """)
    
    query("""\
        drop view if exists $localTableRt
    """)

catch y
    println("clean up Exception ",y)
end


""
